<a href="https://colab.research.google.com/github/rvargas42/Value_Growth_Challenge_ETSFactory/blob/main/data/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Extraction: